<a href="https://colab.research.google.com/github/andrew66882011/qss20_slides_activities/blob/main/activities/05_merging_session2_codeexample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [ ]:
import pandas as pd
import re 
import numpy as np
import datetime
from datetime import datetime

## a couple recordlinkage packages
import fuzzywuzzy
import recordlinkage

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:

## nltk for string distance
import nltk

## jarowinkler
from pyjarowinkler import distance

# Load and view dataset 1: tax certificates for San Diego businesses

In [ ]:
## general link: https://data.sandiego.gov/datasets/business-listings/

## active tax certificates
sd = pd.read_csv("https://seshat.datasd.org/ttcs/sd_businesses_active_datasd.csv")
sd.head()

## PPP loans for CA

## Load and view dataset 2: PPP loans > 150k

General link: https://data.sba.gov/dataset/ppp-foia/resource/3d28c417-5170-4f1f-be31-b0c7b0182501 
        

For a real application, we'd want to programmatically load and rowbind the different < 150k sheets. For this exercise,
we'll just look at the larger loans (>150k) and subset to california

In [ ]:
ppp = pd.read_csv("https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/6b62a44b-69ec-436a-9b95-0ea550475543/download/public_150k_plus.csv")

In [ ]:

## look at address fields to see whether state is relatively complete
## see that state is only missing about 14 so we (1) subset to CA and 
## (2) subset to State == CA and also
## zips that overlap with ones in the SD tax certificate data
## just using raw-zip but to be more careful, if doing for real,
## we'd want to standardize to either 6 dig zip or 10-dig zip
ppp.BorrowerState.value_counts(dropna = False)
ppp = ppp[(ppp.BorrowerState == "CA") &
                  (ppp.BorrowerZip.isin(sd.address_zip))].copy()
ppp.shape


## Step 1 - what are the possible join fields between the two:

San Diego tax certicate:

- Business-level fields:
    - Owner name
    - Business name (dba_name)
    
- Sector-level fields:
    - naics_sector 
    - naics_code
    - naics_description
    
- Geographic fields:
    - City and state 
    - Zip 
    - Bid (business improvement district)
    - Council district
    - Address
    
PPP loans:

- Business-level:
    - BorrowerName
    - Borrower Address
    - BorrowerCity
    - BorrowerState
    - BorrowerZip


## Step 2-- build our matching approach using some manual examples

Examples of two PPP loan recipients:

- THE KLEINFELDER GROUP, INC.
- DURAN FREIGHT CORPORATION

In [ ]:
klein_patt = r".*(\s+)?KLEINFELDER\s+.*"
klein_possible = [biz for biz in sd.dba_name
                 if re.match(klein_patt, biz) is not None]
klein_possible

In [ ]:
duran_patt = r".*(\s+)?DURAN\s+.*"
duran_possible = [biz for biz in sd.dba_name
                 if re.match(duran_patt, biz) is not None]
duran_possible

### Investigate fields that could help weed out false matches for the first business

In [ ]:
sd.columns
ppp.columns

In [ ]:
## defining helpful fields w/in each df for adjudicating matches
ppp_helpfulfields = ["BorrowerName", "BorrowerAddress", "BorrowerCity", 
                    "BorrowerZip", "FranchiseName", "NAICSCode", "ProjectZip"]
sd_helpfulfields = ["dba_name", "naics_code", "naics_sector",
                    "address_no", "address_pd", "address_road",
                    "address_sfx", "address_city", "address_zip", "date_cert_effective",
                   "date_cert_expiration", "business_owner_name"]

In [ ]:
print(ppp.loc[ppp.BorrowerName == "THE KLEINFELDER GROUP, INC.",
                  ppp_helpfulfields])


print(sd.loc[sd.dba_name.isin(klein_possible),
                       sd_helpfulfields])

## see that likely either kleinfelder construction services
## or kleinfelder inc (could match to both); possible slight 
## pref for matching to kleinfelder inc owner name 

### Preview of activity step 1: clean addresses in each of the datasets

Previous example shows us address can help adjudicate b/t matches.

When we break into groups, you'll
    
- Create a new zip code col that's just the first 6 digits
- Paste together the address_no, address_pd, address_road, address_sfx fields in the SD active biz to create a field similar to BorrowerAddress in the PPP loan data (pay attention to capitalization; might be easier to capitalize in each)



# Constructing our own matching function

The package we'll review makes matching easier by putting a lot of the hard stuff under the hood

But it's good to know what's going on under that hood.

Here, using the example of THE KLEINFELDER GROUP, INC., we'll look within the tentative matches + a random other sample of the SD business data to construct match points

## Step 0: pool of sd businesses to look in

Normally we'd look in full set but this helps with runtime

In [ ]:
sd = pd.concat([sd[sd.dba_name.isin(klein_possible)].copy(),
                         sd[~sd.dba_name.isin(klein_possible)].sample(n = 15, 
                        random_state = 922).copy(),
                          sd[sd.dba_name == "DURAN FREIGHT CORPORATION"]])

sd.head()

## Step 1: find string similarity between (1) our focal PPP business (Kleinfelder) and (2) the businesses in the SD pool

Here, we're using Jaccard distance --- common one in addition to that is Jaro Winkler string similarity

Some options here: https://python.gotrained.com/nltk-edit-distance-jaccard-distance/

Can also use fuzzywuzzy installed on jhub- discussion here: https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe

In [ ]:
## first, let's process the biz name
## and remove everything that's not (^)
## words or spaces and also remove the
focal_ppp_raw = "THE KLEINFELDER GROUP, INC."
focal_ppp_cleaner = re.sub("THE\s", 
                           "", 
                    re.sub(r"[^\w\s]", "", focal_ppp_raw))
focal_ppp_cleaner

In [ ]:
### look at a few different distance metrics
sd['dist_focal_edit'] = [nltk.edit_distance(focal_ppp_cleaner, other_name)
                     for other_name in sd.dba_name]

sd[['dba_name', 'dist_focal_edit']].sort_values(by = 'dist_focal_edit')

sd['dist_focal_jacc'] = [nltk.jaccard_distance(set(focal_ppp_cleaner), set(other_name))
                     for other_name in sd.dba_name]

sd[['dba_name', 'dist_focal_jacc']].sort_values(by = 'dist_focal_jacc')


In [ ]:
## jaro is similarity score so 1 - that
sd['dist_focal_jaro'] = [1-distance.get_jaro_distance(focal_ppp_cleaner, other_name,
                                                              winkler = True, scaling = 0.1)
                     for other_name in sd.dba_name]

sd[['dba_name', 'dist_focal_jaro']].sort_values(by = 'dist_focal_jaro')

### Step 2-- rule out potential matches with different zip codes

"Blocking" on 6-digit zip code, or requiring an exact match

In [ ]:
## first, we clean up the SD zip codes to only be 6 dig since we know our focal ppp biz
## has a 6-dig zip code
sd['zip_6dig'] = sd.address_zip.str.replace("\-.*", "", regex = True)

## get the zip- using iloc since we just want it as a string
## rather than series
focal_ppp_zip = ppp.BorrowerZip[ppp.BorrowerName == "THE KLEINFELDER GROUP, INC."].iloc[0]
focal_ppp_zip



In [ ]:
## create true false if same as focal biz
sd['is_match_zip'] = np.where(sd.zip_6dig == focal_ppp_zip,
                                        True, False)

sd.loc[sd.is_match_zip,
             sd_helpfulfields]

### Step 3: construct some match score

Record linkage methods have different ways for aggregating across fields

Here, we're going with a simple one of:

- Need to match the zip code of the focal Kleinfelder group directly
- Within those, find the average of the jarowinkler and jaccard string distance measures (we're excluding edit distance from that avg since on diff scale)

Whichever has the lowest average of two we consider the best match

In [ ]:
string_dist_fields = [col for col in sd.columns if "dist_" in col and 
                     "edit" not in col]
string_dist_fields
mean_distances = sd[string_dist_fields].mean(axis = 1)

mean_distances[0:5]

sd['mean_string_dist'] = mean_distances

sd.loc[sd.is_match_zip,
                 sd_helpfulfields + ['mean_string_dist']].sort_values(by = "mean_string_dist")

## would go with kleinfelder inc and maybe also
## the construction services

# That was a lot of steps. How can we use a package to automate a bit?

Google "fuzzy matching" or "probablistic record linkage" packages in python

Here, we'll focus on 

- recordlinkage. Documentation: https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html



## Step 1. Define dataframes to match

Here, we'll use two dataframes:

- The sd_lookin dataframe we've been working with 
- A ppp dataframe with (1) our focal business, (2) a small random sample of others, (3) the biz we know has an exact match

In [ ]:
## subset of ppp
## to help with runtime
ppp = pd.concat([ppp[ppp.BorrowerName == focal_ppp_raw].copy(),
                       ppp[ppp.BorrowerName != focal_ppp_raw].sample(n = 10, random_state = 42),
                       ppp[ppp.BorrowerName == "DURAN FREIGHT CORPORATION"]])


## clean name similarly to how we did before
ppp['bname_clean'] = [re.sub(r"[^\w\s]", "", one_n) for one_n in ppp.BorrowerName]
ppp[['BorrowerName', 'bname_clean']].head()

## clean zip so that 6 digits
ppp['zip_4match'] = ppp.BorrowerZip.astype(str).str.replace("\-.*", "", regex = True)
ppp[['BorrowerZip', 'zip_4match']].head()

## in exercise, you'll clean address and naics codes 


## Step 2: for ease of use, standardize colnames for the fields we'll use

In this practice exercise, we'll use:

- Fuzzy match on business name
- Exact match on 6-digit zip code

We only need to standardize the name of the exact match field, but are here just standardizing all for ease of use

In [ ]:
## define rename dictionary for sd_biz and rename saving to new (just for convenience to not reload if we want to
## change earlier step)
newcols_sd = {'dba_name': 'bizname_4match',
           'zip_6dig': 'zip_4match'}

sd = sd.rename(columns = newcols_sd, inplace = False)

sd[[col for col in sd.columns if "4match" in col]].head()


## same for ppp data
newcols_ppp = {'bname_clean': 'bizname_4match'}

ppp = ppp.rename(columns = newcols_ppp, inplace = False)

ppp[[col for col in ppp.columns if "4match" in col]].head()


## Step 3: initialize the match object and tell it if anything to "block on" or exact match

Here, we're blocking on zip

In [ ]:
## initialize indexer
my_recordmatcher = recordlinkage.Index()
print(type(my_recordmatcher))

## tell it what to block on (skip if not blocking on anything)
my_recordmatcher.block("zip_4match")



## Step 4: create candidate links based on that blocking variable

In [ ]:
## then, feed the record matcher the two datasets (must have that blocking variable)
## this will create candidate_links that are exact matches on those
candidate_links_zip = my_recordmatcher.index(sd, ppp)
candidate_links_zip

print(type(candidate_links_zip))

## see that it's a list of tuples and first element in tuple is index
## of first df we feed it; second is index in second df we feed it

## print example of links
sd.loc[sd.index == 31421,
         [col for col in sd.columns if "4match" in col]]
ppp.loc[ppp.index.isin([80338, 88795]),
        [col for col in ppp.columns if "4match" in col]]

## Step 5- initialize Compare and define fuzzy fields and threshold for each

Note in documentation about diff string compare methods:
This class is used to compare string values. The implemented algorithms are: ‘jaro’,’jarowinkler’, ‘levenshtein’, ‘damerau_levenshtein’, ‘qgram’ or ‘cosine’. In case of agreement, the similarity is 1 and in case of complete disagreement it is 0. The Python Record Linkage Toolkit uses the jellyfish package for the Jaro, Jaro-Winkler, Levenshtein and Damerau- Levenshtein algorithms.

In [ ]:
compare = recordlinkage.Compare()

thres_bizname = 0.7
compare.string('bizname_4match', 'bizname_4match', method='jaro', threshold=thres_bizname)

print(type(compare))

## Step 6- using the compare Class and the candidate links, compute comparisons

In [ ]:
compare_vectors = compare.compute(candidate_links_zip, sd, ppp)
print(type(compare_vectors))

compare_vectors

## returns result from comparing each pair of records - so we see that with the 2497
## example above (kleinfield construction with naics 54161), 
## which has candidate pairs of (1) Kneinfelder group naics code 541330 (index 34514)
## and (2) globe haru naics code 722511 (index 112928), there seems to be a match on name
## with the first in the pair

## Step 7. decide what counts as a true match

Three general approaches:

- Threshold based: look at the raw scores and determine what scores are above a threshold
- Unsupervised: something that clusters the pairs into "likely match" or "likely not match" but where we're not feeding it "labels" corresponding to true matches
- Supervised: we have some gold-standard label dataset that has an indicator for whether records are true matches; we train a model on those true matches and generalize to new cases

See here for many classifiers: https://recordlinkage.readthedocs.io/en/latest/ref-classifiers.html

Here, we're using unsupervised and k-means clustering algorithm

Other option is an EM-based classifier initialized as follows, but not enough data here to fit:
ecm = recordlinkage.ECMClassifier()
  


In [ ]:
## initialize classifier
kmeans = recordlinkage.KMeansClassifier()
kmeans_results = kmeans.fit_predict(compare_vectors)
print(type(kmeans_results))
kmeans_results


## Step 8- extract pairs using indices and summarize

In [ ]:
## since sd was our left hand side data, they're 
## the first index in the tuple- extract
indices_sd = [x[0] for x in kmeans_results]

## since ppp loans were our right hand side data, they're
## the second index in the tuple - extract
indices_ppp = [x[1] for x in kmeans_results]

## create dataframe
df_matchpairs = pd.DataFrame({'sd_indices': indices_sd,
                'ppp_indices': indices_ppp})

df_matchpairs

## add indices as col to orig data
sd['index_4merge'] = sd.index
ppp['index_4merge'] = ppp.index

## then, join matches

### first, i'm joining the sd info
df_matchpairs_wsd = pd.merge(df_matchpairs,
                            sd[['index_4merge', 'bizname_4match',
                                      'zip_4match']],
                            how = "left",
                            left_on = "sd_indices",
                            right_on = "index_4merge")

df_matchpairs_wsd

## then, i'm joining the ppp info and adding a suffix to distinguish the vars
df_matchpairs_wboth = pd.merge(df_matchpairs_wsd,
                              ppp[['index_4merge', 'bizname_4match',
                                         'zip_4match']],
                              how = "left",
                              left_on = "ppp_indices",
                              right_on = "index_4merge",
                              suffixes= ["_sd_tax", "_ppp"])

df_matchpairs_wboth
